# Clean NSQIP Dataset

This script takes the raw NSQIP dataset ('Pittumor compile working.xlxs') and prepares it for an ML pipeline by fixing encoding, normalizing, and filtering the data.  
  
  V1: 06/6/20  
  V2: 6/27/20  
  Justin M. Campbell, justin.campbell@hsc.utah.edu

### Import Libraries & Data

In [2]:
import sys
import numpy as np
import pandas as pd
import sklearn
import sklearn.preprocessing as preprocessing

dataset = pd.read_csv('/Users/justincampbell/Library/Mobile Documents/com~apple~CloudDocs/Research/Karsy/NSQIP PitTumor ML/Pittumor compile working.csv');


print('Python: {}'.format(sys.version))
print('Pandas: {}'.format(pd.__version__))
print('Numpy: {}'.format(np.__version__))
print('Sklearn: {}'.format(sklearn.__version__))

Python: 3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Pandas: 1.0.1
Numpy: 1.18.1
Sklearn: 0.22.1


### Define Variables of Interest & Filter Dataset

In [3]:
# Variables used in Josh's analysis
cont_vars = ['AGE', 'WEIGHT', 'PUFYEAR', 'OPTIME', 'TOTHLOS', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT']
yesno_vars = ['READMISSION1', 'SMOKE', 'HXCOPD', 'HYPERMED', 'DISCANCR','WNDINF', 'STEROID', 'WTLOSS', 'BLEEDIS','EMERGNCY', 'RETURNOR']
cat_vars = ['SEX', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'DIABETES', 'DYSPNEA', 'PRSEPIS', 'ASACLAS', 'TRANST', 'DISCHDEST']
all_vars = cont_vars + yesno_vars + cat_vars

# Filter dataset to variables of interest, sanity-check
print(len(all_vars))
dataset = dataset[all_vars]
print(dataset.shape)
dataset.head(1)

33
(2292, 33)


,AGE,WEIGHT,PUFYEAR,OPTIME,TOTHLOS,PRSODM,PRBUN,PRCREAT,PRALBUM,PRPLATE,...,RETURNOR,SEX,RACE_NEW,ETHNICITY_HISPANIC,DIABETES,DYSPNEA,PRSEPIS,ASACLAS,TRANST,DISCHDEST
0,78.0,163,2018.0,128,11,138,16.0,0.8,3.5,162.0,...,No,female,White,N,NO,No,None,3-Severe Disturb,Not transferred (admitted from home),Home


### Replace Yes-No Encoding With 1-0

In [4]:
yesno_dataset = dataset[yesno_vars].copy()
yesno_dataset = yesno_dataset.replace({'Yes': 1, 'No': 0})
yesno_dataset - yesno_dataset.drop('BLEEDIS', axis=1) # redundant

,BLEEDIS,DISCANCR,EMERGNCY,HXCOPD,HYPERMED,READMISSION1,RETURNOR,SMOKE,STEROID,WNDINF,WTLOSS
0,NaN,0,0,0,0,NaN,0.0,0,0,0,0
1,NaN,0,0,0,0,NaN,0.0,0,0,0,0
2,NaN,0,0,0,0,0.0,0.0,0,0,0,0
3,NaN,0,0,0,0,NaN,0.0,0,0,0,0
4,NaN,0,0,0,0,NaN,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2287,NaN,0,0,0,0,NaN,0.0,0,0,0,0
2288,NaN,0,0,0,0,NaN,0.0,0,0,0,0
2289,NaN,0,0,0,0,NaN,0.0,0,0,0,0
2290,NaN,0,0,0,0,NaN,0.0,0,0,0,0


In [5]:
bleed = yesno_dataset['BLEEDIS'].copy()
bleed.replace(np.NaN, 2.0, inplace=True)
encode = preprocessing.OneHotEncoder()

In [6]:
encode.fit(bleed.values.reshape(-1,1))
columns = encode.get_feature_names(['BLEEDIS'])
new = encode.transform(bleed.values.reshape(-1,1)).toarray()
bleedis = pd.DataFrame(new, columns=columns )

In [7]:
yesno_dataset = pd.concat([yesno_dataset,bleedis], axis=1)

### Change Categorical Encoding

In [8]:
cat_dataset = dataset[cat_vars].copy()

# Binarizing
cat_dataset['ASACLAS'] = cat_dataset['ASACLAS'].replace({'None assigned': 0, '1-No Disturb': 1, '2-Mild Disturb': 2, '3-Severe Disturb': 3, '4-Life Threat': 4})
cat_dataset['DIABETES'] = cat_dataset['DIABETES'].replace({'NO': 0, 'NON-INSULIN': 1, 'INSULIN': 1})
cat_dataset['DYSPNEA'] = cat_dataset['DYSPNEA'].replace({'No': 0, 'MODERATE EXERTION': 1, 'AT REST': 1})
cat_dataset['PRSEPIS'] = cat_dataset['PRSEPIS'].replace({'None': 0, 'Sepsis': 1, 'SIRS': 1})

# Dummy-encoding
sex = pd.get_dummies(cat_dataset['SEX'], prefix='SEX')
race = pd.get_dummies(cat_dataset['RACE_NEW'], prefix='RACE')
hisp = pd.get_dummies(cat_dataset['ETHNICITY_HISPANIC'], prefix='HISPANIC')
transt = pd.get_dummies(cat_dataset['TRANST'], prefix='TRAN')
disch = pd.get_dummies(cat_dataset['DISCHDEST'], prefix='DISCHDEST')

# Adjust ASA-class formatting
cat_dataset['ASACLAS'] = (cat_dataset['ASACLAS']-cat_dataset['ASACLAS'].min())/(cat_dataset['ASACLAS'].max()-cat_dataset['ASACLAS'].min())

### Pull Continuous Variables

In [9]:
cont_dataset = dataset[cont_vars].copy()

In [10]:
# Merge new variables to produce clean dataset
clean_dataset = pd.concat([cont_dataset, yesno_dataset, cat_dataset, sex, race, hisp, transt, disch], axis=1)

# Remove redundant variables
clean_dataset = clean_dataset.drop(['SEX', 'RACE_NEW', 'BLEEDIS', 'ETHNICITY_HISPANIC', 'TRANST', 'DISCHDEST'], axis=1)

print(clean_dataset.shape)
clean_dataset.head()

(2292, 50)


,AGE,WEIGHT,PUFYEAR,OPTIME,TOTHLOS,PRSODM,PRBUN,PRCREAT,PRALBUM,PRPLATE,...,HISPANIC_Y,TRAN_From acute care hospital inpatient,TRAN_Not transferred (admitted from home),TRAN_Nursing home - Chronic care - Intermediate care,TRAN_Outside emergency department,TRAN_Transfer from other,DISCHDEST_Home,DISCHDEST_Rehab,"DISCHDEST_Skilled Care, Not Home",DISCHDEST_Unknown
0,78.0,163,2018.0,128,11,138,16.0,0.80,3.5,162.0,...,0,0,1,0,0,0,1,0,0,0
1,59.0,233,2018.0,173,3,140,19.0,0.95,4.4,203.0,...,0,0,1,0,0,0,1,0,0,0
2,64.0,157,2018.0,101,2,143,20.0,0.89,4.4,282.0,...,0,0,1,0,0,0,1,0,0,0
3,26.0,218,2018.0,188,4,144,8.0,0.80,4.1,448.0,...,0,0,1,0,0,0,1,0,0,0
4,29.0,138,2018.0,147,2,140,9.0,0.56,-99.0,236.0,...,1,0,1,0,0,0,1,0,0,0


### Replace NaN's With Median Value

In [11]:
# Replace missing values with median values
for col in clean_dataset.columns:
    clean_dataset[col].replace(-99, clean_dataset[col].median(), inplace=True)
    
clean_dataset['AGE'].replace(np.NaN, clean_dataset['AGE'].median(), inplace=True)

# Erop rows that don't have readmission defined
clean_dataset = clean_dataset[clean_dataset['READMISSION1'].notna()]

for col in clean_dataset.columns:
    print('{}: {}'.format(col, clean_dataset[col].count()))

AGE: 1534
WEIGHT: 1534
PUFYEAR: 1534
OPTIME: 1534
TOTHLOS: 1534
PRSODM: 1534
PRBUN: 1534
PRCREAT: 1534
PRALBUM: 1534
PRPLATE: 1534
PRPTT: 1534
PRINR: 1534
PRPT: 1534
READMISSION1: 1534
SMOKE: 1534
HXCOPD: 1534
HYPERMED: 1534
DISCANCR: 1534
WNDINF: 1534
STEROID: 1534
WTLOSS: 1534
EMERGNCY: 1534
RETURNOR: 1534
BLEEDIS_0.0: 1534
BLEEDIS_1.0: 1534
BLEEDIS_2.0: 1534
DIABETES: 1534
DYSPNEA: 1534
PRSEPIS: 1534
ASACLAS: 1533
SEX_female: 1534
SEX_male: 1534
RACE_American Indian or Alaska Native: 1534
RACE_Asian: 1534
RACE_Black or African American: 1534
RACE_Native Hawaiian or Pacific Islander: 1534
RACE_Unknown/Not Reported: 1534
RACE_White: 1534
HISPANIC_N: 1534
HISPANIC_U: 1534
HISPANIC_Y: 1534
TRAN_From acute care hospital inpatient: 1534
TRAN_Not transferred (admitted from home): 1534
TRAN_Nursing home - Chronic care - Intermediate care: 1534
TRAN_Outside emergency department: 1534
TRAN_Transfer from other: 1534
DISCHDEST_Home: 1534
DISCHDEST_Rehab: 1534
DISCHDEST_Skilled Care, Not Home: 1

### Normalize Continuous Data With StandardScaler

In [12]:
# Remove cont vars, standardize, then add back-in
cont_dataset = clean_dataset[cont_vars].copy()
clean_dataset = clean_dataset.drop(cont_vars, axis=1)

scaler = preprocessing.StandardScaler().fit(cont_dataset)
stand_dataset = scaler.transform(cont_dataset)
stand_dataset = pd.DataFrame(stand_dataset)
stand_dataset.columns = cont_vars

In [13]:
merged_dataset = pd.concat([clean_dataset.reset_index(),stand_dataset.reset_index()], axis=1)
print(merged_dataset.shape)
merged_dataset.head()

(1534, 52)


,index,READMISSION1,SMOKE,HXCOPD,HYPERMED,DISCANCR,WNDINF,STEROID,WTLOSS,EMERGNCY,...,OPTIME,TOTHLOS,PRSODM,PRBUN,PRCREAT,PRALBUM,PRPLATE,PRPTT,PRINR,PRPT
0,2,1.0,0,0,1,0,0,0,0,0,...,-0.632849,-0.432784,1.100094,0.696080,-0.003698,1.175803,0.520203,-0.010827,-0.329540,-0.211986
1,30,1.0,0,0,0,0,0,0,0,0,...,2.696533,0.608496,-2.324180,0.111763,0.027710,1.171875,-1.066267,0.292843,1.244182,-0.211986
2,37,1.0,0,0,0,0,0,1,0,0,...,-0.462880,0.013479,0.788797,1.426476,1.598134,-0.855191,-2.046145,-0.536410,0.807037,-0.211986
3,96,1.0,0,0,1,0,0,0,0,0,...,-0.542865,-0.581538,2.656582,0.696080,1.346866,-0.855191,0.442435,-0.805041,-0.941543,-0.211986
4,116,1.0,0,0,1,0,0,0,0,0,...,-0.712834,-0.135275,0.166201,0.403921,0.027710,-0.855191,-0.490783,0.619872,0.020176,-0.211986


### Save

In [14]:
merged_dataset.to_csv('/Users/justincampbell/Library/Mobile Documents/com~apple~CloudDocs/Research/Karsy/NSQIP PitTumor ML/NSQIP_Clean2.csv')